# Old GDP Real-Time Dataset

**Author:** Jason Cruz  
**Last updated:** 08/13/2025  
**Python version:** 3.12  
**Project:** Rationality and Nowcasting on Peruvian GDP Revisions  

---
## 📌 Summary
This notebook documents the step-by-step **construction of real-time datasets** for **Peruvian GDP revisions** since 2013–PRESENT. It covers:

1. **Downloading PDFs** (actually Weekly Reports (WR)) from the Central Reserve Bank of Peru's website.
2. **Generating PDF inputs** by shorten them in order to keep key pages containing required tables where GDP growth rates are in.
3. **Cleaning-up data** extracted from input PDFs.
4. **Concatenating real-time datasets across years by frequency** 
5. **Storing RTD to SQL** for availability to users upon request and further analysis.

🌐 **Main Data Source:** [BCRP Weekly Report](https://www.bcrp.gob.pe/publicaciones/nota-semanal.html) (📰 WR, from here on)  
Any questions or issues regarding the coding, please email [Jason 📨](mailto:jj.cruza@up.edu.pe)  

---

## 🛠️ Libraries

If you don't have the libraries below, please use the following code (as example) to install the required libraries.

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

Check out Python information

In [1]:
import sys
import platform

print("🐍 Python Information")
print(f"  Version  : {sys.version.split()[0]}")
print(f"  Compiler : {platform.python_compiler()}")
print(f"  Build    : {platform.python_build()}")
print(f"  OS       : {platform.system()} {platform.release()}")

🐍 Python Information
  Version  : 3.12.1
  Compiler : MSC v.1916 64 bit (AMD64)
  Build    : ('main', 'Jan 19 2024 15:44:08')
  OS       : Windows 10


**Import helper functions**

> ⚠️ Please, check the script `new_gdp_datasets_functions.py` which contains all the functions required by this _jupyter notebook_. The functions there are ordered according to the sections of this jupyter notebok.

In [2]:
from gdp_rtd_pipeline import *

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


## ⚙️ Initial set-up

Before preprocessing new GDP releases data, we will:

* **Create necessary folders** for storing inputs, outputs, logs, and screenshots.
* **Connect to the PostgreSQL database** containing GDP revisions datasets.
* **Import helper functions** from `new_gdp_datasets_functions.py`.

**Create necessary folders**

In [3]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()
user_input = input("Enter relative path (default='.'): ").strip() or "."
target_path = (PROJECT_ROOT / user_input).resolve()
target_path.mkdir(parents=True, exist_ok=True)
print(f"📂 Using path: {target_path}")

Enter relative path (default='.'):  .


📂 Using path: C:\Users\Jason Cruz\OneDrive\Documentos\RA\CIUP\GDP Revisions\GitHub\peru_gdp_revisions\gdp_revisions_datasets


<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Data cleaning</span></h1>

In [6]:
# Define base folder for saving vintages data (.csv)
old_wr_folder = 'old_wr'

# Define subfolder for saving 
old_wr_subfolder_1 = os.path.join(old_wr_folder, 'table_1')

# Define subfolder for saving 
old_wr_subfolder_2 = os.path.join(old_wr_folder, 'table_2')

# Define base folder for saving vintages data (.csv)
data_folder = 'data'

# Define subfolder for saving 
input_data_subfolder = os.path.join(data_folder, 'input')

# Define subfolder for saving 
output_data_subfolder = os.path.join(data_folder, 'output')

# Create all required folders (if they do not already exist) and confirm creation
for folder in [old_wr_folder, old_wr_subfolder_1, old_wr_subfolder_2, data_folder, input_data_subfolder, output_data_subfolder]:
    os.makedirs(folder, exist_ok=True)
    print(f"📂 {folder} created")

📂 old_wr created
📂 old_wr\table_1 created
📂 old_wr\table_2 created
📂 data created
📂 data\input created
📂 data\output created


In [ ]:
raw_1, clean_1, vintages_1 = table_1_cleaner(
    input_pdf_folder = input_pdf_subfolder,
    record_folder = record_folder,
    record_txt = '3_created_vintages_tab_1.txt',
    persist = True,
    persist_folder = input_data_subfolder,
    pipeline_version = "s3.0.0",
)

In [ ]:
raw_1.keys()

In [ ]:
clean_1.keys()

In [ ]:
vintages_1.keys()

In [ ]:
raw_1['ns_04_2022_1']

In [ ]:
clean_1['ns_04_2022_1']

In [ ]:
vintages_1['ns_04_2022_1']

# Checking the cleaning version out

In [ ]:
df100 = vintages_1["ns_04_2022_1"]
print(df100.attrs)
# {'pipeline_version': 's3.0.0'}


In [ ]:
vintages_1["ns_04_2022_1"].attrs

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Extracting tables and data cleanup
    </span>
    </h2>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
PENDING INTRO TEXT
</p>
<div/>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 1.</span> Extraction and cleaning of data from tables on monthly real GDP growth rates.
    </span>
    </h3>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <b>"ns_dates"</b> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

In [16]:
import locale
import os
import re
import pandas as pd
from tkinter import Tk, messagebox

In [17]:
# Set the locale to Spanish
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Dictionary to store generated DataFrames
old_dataframes_dict_1 = {}

# Path for the processed folders log file
record_path = 'record/old_processed_folders_1.txt'

# Function to correct month names
def correct_month_name(month):
    months_mapping = {
        'setiembre': 'septiembre',
        # Add more mappings as needed for other month names
    }
    return months_mapping.get(month, month)

# Function to register processed folder
def register_processed_folder(folder, num_processed_files):
    with open(record_path, 'a') as file:
        file.write(f"{folder}:{num_processed_files}\n")

# Function to check if folder has been processed
def folder_processed(folder):
    if not os.path.exists(record_path):
        return False
    with open(record_path, 'r') as file:
        for line in file:
            if line.startswith(folder):
                return True
    return False

def process_csv(csv_path):
    old_tables_dict_1 = {}  # Local dictionary for each CSV
    table_counter = 1

    filename = os.path.basename(csv_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        wr, year = id_ns_year_matches[0]
    else:
        print("No matches found for wr and year in filename:", filename)
        return None, None, None  # Return None for tables_dict_1 as well

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    dataframe_name = f"{new_filename}_{table_counter}"
    old_tables_dict_1[dataframe_name] = df.copy()

    # Apply cleanup functions to a copy of the DataFrame
    df_clean = df.copy()

    # Conditional cleaning based on the first column
    if df_clean.columns[1] == 'economic_sectors':
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)
    else:
        # Cleaning functions (set as required)
        df_clean = clean_column_names(df_clean)
        df_clean = adjust_column_names(df_clean)
        df_clean = drop_rare_caracter_row(df_clean)
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        df_clean = reset_index(df_clean)
        df_clean = remove_digit_slash(df_clean)
        df_clean = replace_var_perc_first_column(df_clean)
        df_clean = replace_var_perc_last_columns(df_clean)
        df_clean = replace_number_moving_average(df_clean)
        df_clean = relocate_last_column(df_clean)
        df_clean = clean_first_row(df_clean)
        df_clean = find_year_column(df_clean)
        year_columns = extract_years(df_clean)
        df_clean = get_months_sublist_list(df_clean, year_columns)
        df_clean = first_row_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)

    # Add the column 'year' to the clean DataFrame
    df_clean.insert(0, 'year', year)
    
    # Add the column 'wr' to the clean DataFrame
    df_clean.insert(1, 'wr', wr)

    # Store cleaned DataFrame in old_dataframes_dict_1
    old_dataframes_dict_1[dataframe_name] = df_clean

    return wr, year, old_tables_dict_1

# Function to process folder
def process_folder(folder):
    print(f"Processing folder {os.path.basename(folder)}")
    csv_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]

    num_csv_processed = 0
    num_dataframes_generated = 0

    table_counter = 1  # Initialize table counter here
    old_tables_dict_1 = {}  # Declare old_tables_dict_1 outside main loop
    
    for csv_file in csv_files:
        wr, year, tables_dict_temp = process_csv(csv_file)

        if tables_dict_temp:
            for dataframe_name, df in tables_dict_temp.items():
                file_name = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                dataframe_name = f"{file_name}_1"
                
                # Store raw DataFrame in old_tables_dict_1
                old_tables_dict_1[dataframe_name] = df.copy()
                
                # Apply cleanup functions to a copy of the DataFrame
                df_clean = df.copy()

                # Conditional cleaning based on the first column
                if df_clean.columns[1] == 'economic_sectors':
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                else:
                    # Cleaning functions (set as required)
                    df_clean = clean_column_names(df_clean)
                    df_clean = adjust_column_names(df_clean)
                    df_clean = drop_rare_caracter_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = replace_var_perc_first_column(df_clean)
                    df_clean = replace_var_perc_last_columns(df_clean)
                    df_clean = replace_number_moving_average(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = find_year_column(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = get_months_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)

                # Add the column 'year' to the clean DataFrame
                df_clean.insert(0, 'year', year)

                # Add the column 'wr' to the clean DataFrame
                df_clean.insert(1, 'wr', wr)

                # Store cleaned DataFrame in old_dataframes_dict_1
                old_dataframes_dict_1[dataframe_name] = df_clean

                print(f'  {table_counter}. The dataframe generated for the {csv_file} file is: {dataframe_name}')
                num_dataframes_generated += 1
                table_counter += 1  # Increment table counter here
        
        num_csv_processed += 1  # Increment number of processed CSV for each CSV in folder

    return num_csv_processed, num_dataframes_generated, old_tables_dict_1

def process_folders():
    csv_folder = old_wr_subfolder_1
    folders = [os.path.join(csv_folder, d) for d in os.listdir(csv_folder) if os.path.isdir(os.path.join(csv_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_1 = {}  # Initialize old_tables_dict_1 here
    
    for folder in folders:
        if folder_processed(folder):
            print(f"Folder {folder} has already been processed.")
            continue
        
        num_csv_processed, num_dataframes_generated, tables_dict_temp = process_folder(folder)
        
        # Update old_tables_dict_1 with values returned from process_folder()
        old_tables_dict_1.update(tables_dict_temp)
        
        register_processed_folder(folder, num_csv_processed)

        # Ask user if they want to continue with next folder
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Ensure the messagebox is in front
        message = f"Process {folder} complete. Processed {num_csv_processed} CSV(s) and generated {num_dataframes_generated} DataFrame(s). Continue with next folder?"
        if not messagebox.askyesno("Continue?", message):
            break
            
    print("Processing completed for all folders.")  # Add a message to indicate completion
    
    return old_tables_dict_1

if __name__ == "__main__":
    old_tables_dict_1 = process_folders()



Processing folder 1994
  1. The dataframe generated for the old_wr\table_1\1994\ns-03-1994.csv file is: ns_03_1994_1
  2. The dataframe generated for the old_wr\table_1\1994\ns-08-1994.csv file is: ns_08_1994_1
  3. The dataframe generated for the old_wr\table_1\1994\ns-11-1994.csv file is: ns_11_1994_1
  4. The dataframe generated for the old_wr\table_1\1994\ns-15-1994.csv file is: ns_15_1994_1
  5. The dataframe generated for the old_wr\table_1\1994\ns-19-1994.csv file is: ns_19_1994_1
  6. The dataframe generated for the old_wr\table_1\1994\ns-24-1994.csv file is: ns_24_1994_1
  7. The dataframe generated for the old_wr\table_1\1994\ns-27-1994.csv file is: ns_27_1994_1
  8. The dataframe generated for the old_wr\table_1\1994\ns-31-1994.csv file is: ns_31_1994_1
  9. The dataframe generated for the old_wr\table_1\1994\ns-36-1994.csv file is: ns_36_1994_1
  10. The dataframe generated for the old_wr\table_1\1994\ns-40-1994.csv file is: ns_40_1994_1
  11. The dataframe generated for th

  38. The dataframe generated for the input_data\table_1\1995\ns-38-1995.csv file is: ns_38_1995_1
  39. The dataframe generated for the input_data\table_1\1995\ns-39-1995.csv file is: ns_39_1995_1
  40. The dataframe generated for the input_data\table_1\1995\ns-40-1995.csv file is: ns_40_1995_1
  41. The dataframe generated for the input_data\table_1\1995\ns-41-1995.csv file is: ns_41_1995_1
  42. The dataframe generated for the input_data\table_1\1995\ns-42-1995.csv file is: ns_42_1995_1
  43. The dataframe generated for the input_data\table_1\1995\ns-43-1995.csv file is: ns_43_1995_1
  44. The dataframe generated for the input_data\table_1\1995\ns-44-1995.csv file is: ns_44_1995_1
  45. The dataframe generated for the input_data\table_1\1995\ns-45-1995.csv file is: ns_45_1995_1
  46. The dataframe generated for the input_data\table_1\1995\ns-46-1995.csv file is: ns_46_1995_1
  47. The dataframe generated for the input_data\table_1\1995\ns-47-1995.csv file is: ns_47_1995_1
  48. The 

  18. The dataframe generated for the input_data\table_1\1997\ns-18-1997.csv file is: ns_18_1997_1
  19. The dataframe generated for the input_data\table_1\1997\ns-19-1997.csv file is: ns_19_1997_1
  20. The dataframe generated for the input_data\table_1\1997\ns-20-1997.csv file is: ns_20_1997_1
  21. The dataframe generated for the input_data\table_1\1997\ns-21-1997.csv file is: ns_21_1997_1
  22. The dataframe generated for the input_data\table_1\1997\ns-22-1997.csv file is: ns_22_1997_1
  23. The dataframe generated for the input_data\table_1\1997\ns-23-1997.csv file is: ns_23_1997_1
  24. The dataframe generated for the input_data\table_1\1997\ns-24-1997.csv file is: ns_24_1997_1
  25. The dataframe generated for the input_data\table_1\1997\ns-25-1997.csv file is: ns_25_1997_1
  26. The dataframe generated for the input_data\table_1\1997\ns-26-1997.csv file is: ns_26_1997_1
  27. The dataframe generated for the input_data\table_1\1997\ns-27-1997.csv file is: ns_27_1997_1
  28. The 

  5. The dataframe generated for the input_data\table_1\1999\ns-05-1999.csv file is: ns_05_1999_1
  6. The dataframe generated for the input_data\table_1\1999\ns-06-1999.csv file is: ns_06_1999_1
  7. The dataframe generated for the input_data\table_1\1999\ns-07-1999.csv file is: ns_07_1999_1
  8. The dataframe generated for the input_data\table_1\1999\ns-08-1999.csv file is: ns_08_1999_1
  9. The dataframe generated for the input_data\table_1\1999\ns-09-1999.csv file is: ns_09_1999_1
  10. The dataframe generated for the input_data\table_1\1999\ns-10-1999.csv file is: ns_10_1999_1
  11. The dataframe generated for the input_data\table_1\1999\ns-11-1999.csv file is: ns_11_1999_1
  12. The dataframe generated for the input_data\table_1\1999\ns-12-1999.csv file is: ns_12_1999_1
  13. The dataframe generated for the input_data\table_1\1999\ns-13-1999.csv file is: ns_13_1999_1
  14. The dataframe generated for the input_data\table_1\1999\ns-14-1999.csv file is: ns_14_1999_1
  15. The dataf

  42. The dataframe generated for the input_data\table_1\2000\ns-42-2000.csv file is: ns_42_2000_1
  43. The dataframe generated for the input_data\table_1\2000\ns-43-2000.csv file is: ns_43_2000_1
  44. The dataframe generated for the input_data\table_1\2000\ns-44-2000.csv file is: ns_44_2000_1
  45. The dataframe generated for the input_data\table_1\2000\ns-45-2000.csv file is: ns_45_2000_1
  46. The dataframe generated for the input_data\table_1\2000\ns-46-2000.csv file is: ns_46_2000_1
  47. The dataframe generated for the input_data\table_1\2000\ns-47-2000.csv file is: ns_47_2000_1
  48. The dataframe generated for the input_data\table_1\2000\ns-48-2000.csv file is: ns_48_2000_1
  49. The dataframe generated for the input_data\table_1\2000\ns-49-2000.csv file is: ns_49_2000_1
Processing folder 2001
  1. The dataframe generated for the input_data\table_1\2001\ns-01-2001.csv file is: ns_01_2001_1
  2. The dataframe generated for the input_data\table_1\2001\ns-02-2001.csv file is: ns

...

  17. The dataframe generated for the input_data\table_1\2004\ns-17-2004.csv file is: ns_17_2004_1
  18. The dataframe generated for the input_data\table_1\2004\ns-18-2004.csv file is: ns_18_2004_1
  19. The dataframe generated for the input_data\table_1\2004\ns-19-2004.csv file is: ns_19_2004_1
  20. The dataframe generated for the input_data\table_1\2004\ns-20-2004.csv file is: ns_20_2004_1
  21. The dataframe generated for the input_data\table_1\2004\ns-21-2004.csv file is: ns_21_2004_1
  22. The dataframe generated for the input_data\table_1\2004\ns-22-2004.csv file is: ns_22_2004_1
  23. The dataframe generated for the input_data\table_1\2004\ns-23-2004.csv file is: ns_23_2004_1
  24. The dataframe generated for the input_data\table_1\2004\ns-24-2004.csv file is: ns_24_2004_1
  25. The dataframe generated for the input_data\table_1\2004\ns-25-2004.csv file is: ns_25_2004_1
  26. The dataframe generated for the input_data\table_1\2004\ns-26-2004.csv file is: ns_26_2004_1
  27. The 

  6. The dataframe generated for the input_data\table_1\2006\ns-06-2006.csv file is: ns_06_2006_1
  7. The dataframe generated for the input_data\table_1\2006\ns-07-2006.csv file is: ns_07_2006_1
  8. The dataframe generated for the input_data\table_1\2006\ns-08-2006.csv file is: ns_08_2006_1
  9. The dataframe generated for the input_data\table_1\2006\ns-09-2006.csv file is: ns_09_2006_1
  10. The dataframe generated for the input_data\table_1\2006\ns-10-2006.csv file is: ns_10_2006_1
  11. The dataframe generated for the input_data\table_1\2006\ns-11-2006.csv file is: ns_11_2006_1
  12. The dataframe generated for the input_data\table_1\2006\ns-12-2006.csv file is: ns_12_2006_1
  13. The dataframe generated for the input_data\table_1\2006\ns-13-2006.csv file is: ns_13_2006_1
  14. The dataframe generated for the input_data\table_1\2006\ns-14-2006.csv file is: ns_14_2006_1
  15. The dataframe generated for the input_data\table_1\2006\ns-15-2006.csv file is: ns_15_2006_1
  16. The data

  45. The dataframe generated for the input_data\table_1\2007\ns-45-2007.csv file is: ns_45_2007_1
  46. The dataframe generated for the input_data\table_1\2007\ns-46-2007.csv file is: ns_46_2007_1
  47. The dataframe generated for the input_data\table_1\2007\ns-47-2007.csv file is: ns_47_2007_1
  48. The dataframe generated for the input_data\table_1\2007\ns-48-2007.csv file is: ns_48_2007_1
  49. The dataframe generated for the input_data\table_1\2007\ns-49-2007.csv file is: ns_49_2007_1
Processing folder 2008
  1. The dataframe generated for the input_data\table_1\2008\ns-01-2008.csv file is: ns_01_2008_1
  2. The dataframe generated for the input_data\table_1\2008\ns-02-2008.csv file is: ns_02_2008_1
  3. The dataframe generated for the input_data\table_1\2008\ns-03-2008.csv file is: ns_03_2008_1
  4. The dataframe generated for the input_data\table_1\2008\ns-04-2008.csv file is: ns_04_2008_1
  5. The dataframe generated for the input_data\table_1\2008\ns-05-2008.csv file is: ns_05

  32. The dataframe generated for the input_data\table_1\2009\ns-32-2009.csv file is: ns_32_2009_1
  33. The dataframe generated for the input_data\table_1\2009\ns-33-2009.csv file is: ns_33_2009_1
  34. The dataframe generated for the input_data\table_1\2009\ns-34-2009.csv file is: ns_34_2009_1
  35. The dataframe generated for the input_data\table_1\2009\ns-35-2009.csv file is: ns_35_2009_1
  36. The dataframe generated for the input_data\table_1\2009\ns-36-2009.csv file is: ns_36_2009_1
  37. The dataframe generated for the input_data\table_1\2009\ns-37-2009.csv file is: ns_37_2009_1
  38. The dataframe generated for the input_data\table_1\2009\ns-38-2009.csv file is: ns_38_2009_1
  39. The dataframe generated for the input_data\table_1\2009\ns-39-2009.csv file is: ns_39_2009_1
  40. The dataframe generated for the input_data\table_1\2009\ns-40-2009.csv file is: ns_40_2009_1
  41. The dataframe generated for the input_data\table_1\2009\ns-41-2009.csv file is: ns_41_2009_1
  42. The 

  20. The dataframe generated for the input_data\table_1\2011\ns-20-2011.csv file is: ns_20_2011_1
  21. The dataframe generated for the input_data\table_1\2011\ns-21-2011.csv file is: ns_21_2011_1
  22. The dataframe generated for the input_data\table_1\2011\ns-22-2011.csv file is: ns_22_2011_1
  23. The dataframe generated for the input_data\table_1\2011\ns-23-2011.csv file is: ns_23_2011_1
  24. The dataframe generated for the input_data\table_1\2011\ns-24-2011.csv file is: ns_24_2011_1
  25. The dataframe generated for the input_data\table_1\2011\ns-25-2011.csv file is: ns_25_2011_1
  26. The dataframe generated for the input_data\table_1\2011\ns-26-2011.csv file is: ns_26_2011_1
  27. The dataframe generated for the input_data\table_1\2011\ns-27-2011.csv file is: ns_27_2011_1
  28. The dataframe generated for the input_data\table_1\2011\ns-28-2011.csv file is: ns_28_2011_1
  29. The dataframe generated for the input_data\table_1\2011\ns-29-2011.csv file is: ns_29_2011_1
  30. The 

In [18]:
old_tables_dict_1['ns_51_2002_1'].head(30)

,sectores_economicos,economic_sectors,2001_oct,2001_nov,2001_dic,2001_year,2002_ene,2002_feb,2002_mar,2002_abr,2002_may,2002_jun,2002_jul,2002_ago,2002_set,2002_oct,2002_ene_oct
0,agropecuario,agriculture and livestock,3.6,0.3,0.7,-0.6,7.0,6.6,8.8,11.0,9.3,-0.7,1.6,-0.3,3.7,2.4,5.0
1,agricola,agriculture,5.5,-0.5,-2.9,-2.4,9.0,6.7,10.8,13.5,11.7,-3.9,0.0,-4.0,3.8,2.8,5.3
2,pecuario,livestock,1.8,1.0,4.4,1.7,4.9,6.5,6.2,6.2,3.8,5.9,4.0,3.9,3.6,2.0,4.7
3,pesca,fishing,4.0,-54.1,-38.0,-13.3,-14.6,-25.0,-22.4,-17.0,12.9,37.4,5.4,0.1,18.3,-27.3,-3.9
4,mineria e hidrocarburos,mining and fuel,17.5,23.5,27.9,11.2,29.8,19.5,27.2,19.7,15.2,21.0,0.9,6.9,2.8,4.5,13.8
5,mineria,metals,19.4,26.7,30.8,12.8,32.8,21.1,29.9,22.2,17.7,22.5,1.2,7.7,3.0,5.0,15.1
6,hidrocarburos,fuel,1.3,-4.0,3.6,-2.0,5.3,5.8,4.5,-1.1,-4.5,7.9,-1.5,-1.0,0.3,-1.4,1.2
7,manufactura,manufacturing,3.0,-3.7,-2.1,-1.1,1.7,0.2,-4.1,8.7,1.5,3.1,5.4,4.1,7.4,3.1,3.1
8,procesadores recursos primarios,based on raw materials,8.0,-18.4,-10.9,-2.9,2.2,-4.2,-8.3,-7.1,-3.5,4.5,0.0,-3.3,-1.9,-6.8,-2.9
9,industria no primaria,non primary industries,1.8,0.8,0.6,-0.6,1.6,1.3,-2.9,13.8,3.2,2.8,7.0,6.0,9.7,5.7,4.8


In [19]:
old_dataframes_dict_1['ns_51_2002_1'].head(30)

,year,wr,sectores_economicos,economic_sectors,2001_oct,2001_nov,2001_dic,2001_year,2002_ene,2002_feb,2002_mar,2002_abr,2002_may,2002_jun,2002_jul,2002_ago,2002_sep,2002_oct,2002_ene_oct
0,2002,51,agropecuario,agriculture and livestock,3.6,0.3,0.7,-0.6,7.0,6.6,8.8,11.0,9.3,-0.7,1.6,-0.3,3.7,2.4,5.0
1,2002,51,agricola,agriculture,5.5,-0.5,-2.9,-2.4,9.0,6.7,10.8,13.5,11.7,-3.9,0.0,-4.0,3.8,2.8,5.3
2,2002,51,pecuario,livestock,1.8,1.0,4.4,1.7,4.9,6.5,6.2,6.2,3.8,5.9,4.0,3.9,3.6,2.0,4.7
3,2002,51,pesca,fishing,4.0,-54.1,-38.0,-13.3,-14.6,-25.0,-22.4,-17.0,12.9,37.4,5.4,0.1,18.3,-27.3,-3.9
4,2002,51,mineria e hidrocarburos,mining and fuel,17.5,23.5,27.9,11.2,29.8,19.5,27.2,19.7,15.2,21.0,0.9,6.9,2.8,4.5,13.8
5,2002,51,mineria,metals,19.4,26.7,30.8,12.8,32.8,21.1,29.9,22.2,17.7,22.5,1.2,7.7,3.0,5.0,15.1
6,2002,51,hidrocarburos,fuel,1.3,-4.0,3.6,-2.0,5.3,5.8,4.5,-1.1,-4.5,7.9,-1.5,-1.0,0.3,-1.4,1.2
7,2002,51,manufactura,manufacturing,3.0,-3.7,-2.1,-1.1,1.7,0.2,-4.1,8.7,1.5,3.1,5.4,4.1,7.4,3.1,3.1
8,2002,51,procesadores recursos primarios,based on raw materials,8.0,-18.4,-10.9,-2.9,2.2,-4.2,-8.3,-7.1,-3.5,4.5,0.0,-3.3,-1.9,-6.8,-2.9
9,2002,51,industria no primaria,non primary industries,1.8,0.8,0.6,-0.6,1.6,1.3,-2.9,13.8,3.2,2.8,7.0,6.0,9.7,5.7,4.8


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 2.</span> Extraction and cleaning of data from tables on quarterly and annual real GDP growth rates.
    </span>
    </h3>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <b>"ns_dates"</b> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

In [ ]:
# Set the locale to Spanish
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Dictionary to store generated DataFrames
old_dataframes_dict_2 = {}

# Path for the processed folders log file
record_path = 'dataframes_record/old_processed_folders_2.txt'

# Function to correct month names
def correct_month_name(month):
    months_mapping = {
        'setiembre': 'septiembre',
        # Add more mappings as needed for other month names
    }
    return months_mapping.get(month, month)

# Function to register processed folder
def register_processed_folder(folder, num_processed_files):
    with open(record_path, 'a') as file:
        file.write(f"{folder}:{num_processed_files}\n")

# Function to check if folder has been processed
def folder_processed(folder):
    if not os.path.exists(record_path):
        return False
    with open(record_path, 'r') as file:
        for line in file:
            if line.startswith(folder):
                return True
    return False

# Function to fetch date from database
def get_date(df, engine):
    wr = df['wr'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE wr = '{wr}' AND year = '{year}';"
    date_result = pd.read_sql(query, engine)
    return date_result.iloc[0, 0] if not date_result.empty else None

def process_csv(csv_path, engine):
    old_tables_dict_2 = {}  # Local dictionary for each CSV
    table_counter = 2

    filename = os.path.basename(csv_path)
    wr_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if wr_year_matches:
        wr, year = wr_year_matches[0]
    else:
        print("No matches found for wr and year in filename:", filename)
        return None, None, None, None # Return None for tables_dict_1 as well

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    dataframe_name = f"{new_filename}_{table_counter}"
    old_tables_dict_2[dataframe_name] = df.copy()

    # Apply cleanup functions to a copy of the DataFrame
    df_clean = df.copy()

    # Conditional cleaning based on the first column
    if df_clean.columns[1] == 'economic_sectors':
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)
    else:
        # Cleaning functions (set as required)
        df_clean = replace_total_with_year(df_clean)
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        year_columns = extract_years(df_clean)
        df_clean = roman_arabic(df_clean)
        df_clean = fix_duplicates(df_clean)
        df_clean = relocate_last_column(df_clean)
        df_clean = replace_first_row_nan(df_clean)
        df_clean = clean_first_row(df_clean)
        df_clean = get_quarters_sublist_list(df_clean, year_columns)
        df_clean = reset_index(df_clean)
        df_clean = first_row_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = reset_index(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)

    # Add the column 'year' to the clean DataFrame
    df_clean.insert(0, 'year', year)
    
    # Add the column 'wr' to the clean DataFrame
    df_clean.insert(1, 'wr', wr)
    
    # Get corresponding date from database
    date = get_date(df_clean, engine)
    if date:
        # Add 'date' column to cleaned DataFrame
        df_clean.insert(2, 'date', date)
    else:
        print("Date not found in database for wr:", wr, "and year:", year)

    # Store cleaned DataFrame in old_dataframes_dict_2
    old_dataframes_dict_2[dataframe_name] = df_clean

    return wr, year, old_tables_dict_2

# Function to process folder
def process_folder(folder, engine):
    print(f"Processing folder {os.path.basename(folder)}")
    csv_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]

    num_csv_processed = 0
    num_dataframes_generated = 0

    table_counter = 1  # Initialize table counter here
    old_tables_dict_2 = {}  # Declare old_tables_dict_1 outside main loop
    
    for csv_file in csv_files:
        wr, year, tables_dict_temp = process_csv(csv_file, engine)

        if tables_dict_temp:
            for dataframe_name, df in tables_dict_temp.items():
                file_name = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                dataframe_name = f"{file_name}_2"
                
                # Store raw DataFrame in old_tables_dict_2
                old_tables_dict_2[dataframe_name] = df.copy()
                
                # Procesar y limpiar el DataFrame
                df_clean = df.copy()
                
                # Conditional cleaning based on the first column
                if df_clean.columns[1] == 'economic_sectors':
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                else:
                    # Cleaning functions (set as required)
                    df_clean = replace_total_with_year(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = roman_arabic(df_clean)
                    df_clean = fix_duplicates(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = replace_first_row_nan(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = get_quarters_sublist_list(df_clean, year_columns)
                    df_clean = reset_index(df_clean)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                
                # Add the column 'year' to the clean DataFrame
                df_clean.insert(0, 'year', year)

                # Add the column 'wr' to the clean DataFrame
                df_clean.insert(1, 'wr', wr)

                # Get corresponding date from database
                date = get_date(df_clean, engine)
                if date:
                    # Add 'date' column to cleaned DataFrame
                    df_clean.insert(2, 'date', date)
                else:
                    print("Date not found in database for wr:", wr, "and year:", year)

                # Store cleaned DataFrame in old_dataframes_dict_2
                old_dataframes_dict_2[dataframe_name] = df_clean

                print(f'  {table_counter}. The dataframe generated for the {csv_file} file is: {dataframe_name}')
                num_dataframes_generated += 1
                table_counter += 1  # Increment table counter here
        
        num_csv_processed += 1  # Increment number of processed CSV for each CSV in folder

    return num_csv_processed, num_dataframes_generated, old_tables_dict_2

def process_folders():
    csv_folder = table_2_folder
    folders = [os.path.join(csv_folder, d) for d in os.listdir(csv_folder) if os.path.isdir(os.path.join(csv_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_2 = {}  # Initialize old_tables_dict_1 here
    
    for folder in folders:
        if folder_processed(folder):
            print(f"Folder {folder} has already been processed.")
            continue
        
        num_csv_processed, num_dataframes_generated, tables_dict_temp = process_folder(folder, engine)
        
        # Update old_tables_dict_2 with values returned from process_folder()
        old_tables_dict_2.update(tables_dict_temp)
        
        register_processed_folder(folder, num_csv_processed)
    
        # Ask user if they want to continue with next folder
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Ensure the messagebox is in front
        message = f"Process {folder} complete. Processed {num_csv_processed} CSV(s) and generated {num_dataframes_generated} DataFrame(s). Continue with next folder?"
        if not messagebox.askyesno("Continue?", message):
            break

    print("Processing completed for all folders.")  # Add a message to indicate completion

    return old_tables_dict_2
    
if __name__ == "__main__":
    engine = create_sqlalchemy_engine()
    old_tables_dict_2 = process_folders()

  37. The dataframe generated for the input_data\table_2\1998\ns-37-1998.csv file is: ns_37_1998_2
  38. The dataframe generated for the input_data\table_2\1998\ns-38-1998.csv file is: ns_38_1998_2
  39. The dataframe generated for the input_data\table_2\1998\ns-39-1998.csv file is: ns_39_1998_2
  40. The dataframe generated for the input_data\table_2\1998\ns-40-1998.csv file is: ns_40_1998_2
  41. The dataframe generated for the input_data\table_2\1998\ns-41-1998.csv file is: ns_41_1998_2
  42. The dataframe generated for the input_data\table_2\1998\ns-42-1998.csv file is: ns_42_1998_2
  43. The dataframe generated for the input_data\table_2\1998\ns-43-1998.csv file is: ns_43_1998_2
  44. The dataframe generated for the input_data\table_2\1998\ns-44-1998.csv file is: ns_44_1998_2
  45. The dataframe generated for the input_data\table_2\1998\ns-45-1998.csv file is: ns_45_1998_2
  46. The dataframe generated for the input_data\table_2\1998\ns-46-1998.csv file is: ns_46_1998_2
  47. The 

  23. The dataframe generated for the input_data\table_2\2000\ns-23-2000.csv file is: ns_23_2000_2
  24. The dataframe generated for the input_data\table_2\2000\ns-24-2000.csv file is: ns_24_2000_2
  25. The dataframe generated for the input_data\table_2\2000\ns-25-2000.csv file is: ns_25_2000_2
  26. The dataframe generated for the input_data\table_2\2000\ns-26-2000.csv file is: ns_26_2000_2
  27. The dataframe generated for the input_data\table_2\2000\ns-27-2000.csv file is: ns_27_2000_2
  28. The dataframe generated for the input_data\table_2\2000\ns-28-2000.csv file is: ns_28_2000_2
  29. The dataframe generated for the input_data\table_2\2000\ns-29-2000.csv file is: ns_29_2000_2
  30. The dataframe generated for the input_data\table_2\2000\ns-30-2000.csv file is: ns_30_2000_2
  31. The dataframe generated for the input_data\table_2\2000\ns-31-2000.csv file is: ns_31_2000_2
  32. The dataframe generated for the input_data\table_2\2000\ns-32-2000.csv file is: ns_32_2000_2
  33. The 

  14. The dataframe generated for the input_data\table_2\2002\ns-14-2002.csv file is: ns_14_2002_2
  15. The dataframe generated for the input_data\table_2\2002\ns-15-2002.csv file is: ns_15_2002_2
  16. The dataframe generated for the input_data\table_2\2002\ns-16-2002.csv file is: ns_16_2002_2
  17. The dataframe generated for the input_data\table_2\2002\ns-17-2002.csv file is: ns_17_2002_2
  18. The dataframe generated for the input_data\table_2\2002\ns-18-2002.csv file is: ns_18_2002_2
  19. The dataframe generated for the input_data\table_2\2002\ns-19-2002.csv file is: ns_19_2002_2
  20. The dataframe generated for the input_data\table_2\2002\ns-20-2002.csv file is: ns_20_2002_2
  21. The dataframe generated for the input_data\table_2\2002\ns-21-2002.csv file is: ns_21_2002_2
  22. The dataframe generated for the input_data\table_2\2002\ns-22-2002.csv file is: ns_22_2002_2
  23. The dataframe generated for the input_data\table_2\2002\ns-23-2002.csv file is: ns_23_2002_2
  24. The 

Processing folder 2004
  1. The dataframe generated for the input_data\table_2\2004\ns-01-2004.csv file is: ns_01_2004_2
  2. The dataframe generated for the input_data\table_2\2004\ns-02-2004.csv file is: ns_02_2004_2
  3. The dataframe generated for the input_data\table_2\2004\ns-03-2004.csv file is: ns_03_2004_2
  4. The dataframe generated for the input_data\table_2\2004\ns-04-2004.csv file is: ns_04_2004_2
  5. The dataframe generated for the input_data\table_2\2004\ns-05-2004.csv file is: ns_05_2004_2
  6. The dataframe generated for the input_data\table_2\2004\ns-06-2004.csv file is: ns_06_2004_2
  7. The dataframe generated for the input_data\table_2\2004\ns-07-2004.csv file is: ns_07_2004_2
  8. The dataframe generated for the input_data\table_2\2004\ns-08-2004.csv file is: ns_08_2004_2
  9. The dataframe generated for the input_data\table_2\2004\ns-09-2004.csv file is: ns_09_2004_2
  10. The dataframe generated for the input_data\table_2\2004\ns-10-2004.csv file is: ns_10_200

  38. The dataframe generated for the input_data\table_2\2005\ns-38-2005.csv file is: ns_38_2005_2
  39. The dataframe generated for the input_data\table_2\2005\ns-39-2005.csv file is: ns_39_2005_2
  40. The dataframe generated for the input_data\table_2\2005\ns-40-2005.csv file is: ns_40_2005_2
  41. The dataframe generated for the input_data\table_2\2005\ns-41-2005.csv file is: ns_41_2005_2
  42. The dataframe generated for the input_data\table_2\2005\ns-42-2005.csv file is: ns_42_2005_2
  43. The dataframe generated for the input_data\table_2\2005\ns-43-2005.csv file is: ns_43_2005_2
  44. The dataframe generated for the input_data\table_2\2005\ns-44-2005.csv file is: ns_44_2005_2
  45. The dataframe generated for the input_data\table_2\2005\ns-45-2005.csv file is: ns_45_2005_2
  46. The dataframe generated for the input_data\table_2\2005\ns-46-2005.csv file is: ns_46_2005_2
  47. The dataframe generated for the input_data\table_2\2005\ns-47-2005.csv file is: ns_47_2005_2
  48. The 

  30. The dataframe generated for the input_data\table_2\2007\ns-30-2007.csv file is: ns_30_2007_2
  31. The dataframe generated for the input_data\table_2\2007\ns-31-2007.csv file is: ns_31_2007_2
  32. The dataframe generated for the input_data\table_2\2007\ns-32-2007.csv file is: ns_32_2007_2
  33. The dataframe generated for the input_data\table_2\2007\ns-33-2007.csv file is: ns_33_2007_2
  34. The dataframe generated for the input_data\table_2\2007\ns-34-2007.csv file is: ns_34_2007_2
  35. The dataframe generated for the input_data\table_2\2007\ns-35-2007.csv file is: ns_35_2007_2
  36. The dataframe generated for the input_data\table_2\2007\ns-36-2007.csv file is: ns_36_2007_2
  37. The dataframe generated for the input_data\table_2\2007\ns-37-2007.csv file is: ns_37_2007_2
  38. The dataframe generated for the input_data\table_2\2007\ns-38-2007.csv file is: ns_38_2007_2
  39. The dataframe generated for the input_data\table_2\2007\ns-39-2007.csv file is: ns_39_2007_2
  40. The 

  16. The dataframe generated for the input_data\table_2\2009\ns-16-2009.csv file is: ns_16_2009_2
  17. The dataframe generated for the input_data\table_2\2009\ns-17-2009.csv file is: ns_17_2009_2
  18. The dataframe generated for the input_data\table_2\2009\ns-18-2009.csv file is: ns_18_2009_2
  19. The dataframe generated for the input_data\table_2\2009\ns-19-2009.csv file is: ns_19_2009_2
  20. The dataframe generated for the input_data\table_2\2009\ns-20-2009.csv file is: ns_20_2009_2
  21. The dataframe generated for the input_data\table_2\2009\ns-21-2009.csv file is: ns_21_2009_2
  22. The dataframe generated for the input_data\table_2\2009\ns-22-2009.csv file is: ns_22_2009_2
  23. The dataframe generated for the input_data\table_2\2009\ns-23-2009.csv file is: ns_23_2009_2
  24. The dataframe generated for the input_data\table_2\2009\ns-24-2009.csv file is: ns_24_2009_2
  25. The dataframe generated for the input_data\table_2\2009\ns-25-2009.csv file is: ns_25_2009_2
  26. The 

Processing folder 2011
  1. The dataframe generated for the input_data\table_2\2011\ns-01-2011.csv file is: ns_01_2011_2
  2. The dataframe generated for the input_data\table_2\2011\ns-02-2011.csv file is: ns_02_2011_2
  3. The dataframe generated for the input_data\table_2\2011\ns-03-2011.csv file is: ns_03_2011_2
  4. The dataframe generated for the input_data\table_2\2011\ns-04-2011.csv file is: ns_04_2011_2
  5. The dataframe generated for the input_data\table_2\2011\ns-05-2011.csv file is: ns_05_2011_2
  6. The dataframe generated for the input_data\table_2\2011\ns-06-2011.csv file is: ns_06_2011_2
  7. The dataframe generated for the input_data\table_2\2011\ns-07-2011.csv file is: ns_07_2011_2
  8. The dataframe generated for the input_data\table_2\2011\ns-08-2011.csv file is: ns_08_2011_2
  9. The dataframe generated for the input_data\table_2\2011\ns-09-2011.csv file is: ns_09_2011_2
  10. The dataframe generated for the input_data\table_2\2011\ns-10-2011.csv file is: ns_10_201

  37. The dataframe generated for the input_data\table_2\2012\ns-37-2012.csv file is: ns_37_2012_2
  38. The dataframe generated for the input_data\table_2\2012\ns-38-2012.csv file is: ns_38_2012_2
  39. The dataframe generated for the input_data\table_2\2012\ns-39-2012.csv file is: ns_39_2012_2
  40. The dataframe generated for the input_data\table_2\2012\ns-40-2012.csv file is: ns_40_2012_2
  41. The dataframe generated for the input_data\table_2\2012\ns-41-2012.csv file is: ns_41_2012_2
  42. The dataframe generated for the input_data\table_2\2012\ns-42-2012.csv file is: ns_42_2012_2
  43. The dataframe generated for the input_data\table_2\2012\ns-43-2012.csv file is: ns_43_2012_2
  44. The dataframe generated for the input_data\table_2\2012\ns-44-2012.csv file is: ns_44_2012_2
  45. The dataframe generated for the input_data\table_2\2012\ns-45-2012.csv file is: ns_45_2012_2
  46. The dataframe generated for the input_data\table_2\2012\ns-46-2012.csv file is: ns_46_2012_2
  47. The 

In [ ]:
old_tables_dict_2['ns_44_2004_2']

In [ ]:
old_dataframes_dict_2['ns_44_2004_2']

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Real-time data of Peru's GDP growth rates</span></h1>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
This section creates the GDP growth rate vintages for Peru using <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 1</a> and <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 2</a>, which were extracted and cleaned in the previous section. Each table from each WR (CSV <span style="font-size: 24px;">&#128452;</span>) was extracted and cleaned individually in the previous section. Here, we will concatenate all the tables for a specific economic sector, thus creating a vintage dataset of (real) GDP growth by economic sector from <b>1994</b> to <b>2012</b> (<a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 1</a>) and <b>1997</b> to <b>2012</b> (<a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 2</a>).
<div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
As preferred or as appropriate, you can create the data manually, step by step, or focus on specific sectors or frequencies. Alternatively, you can choose a more efficient or automated approach by generating the data for all sectors and frequencies simultaneously.
<div/>

<div id="3-1">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Manual process of data creation in real time: sector by sector and frequency by frequency.
    </span>
    </h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    With this method you can create and inspect the dataset sector by sector and frequency by frequency. This is useful if you want to create data only for particular sectors and frequencies.
<div/>

<div id="select_sector">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">$\bullet$</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Select <code>sector_economico</code> and <code>economic_sector</code></span></h1>
    </div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
When executing the following code, a window will be displayed with options in <b>Spanish</b> and <b>English</b> to select <b>economic sectors</b>. Choose them to concatenate Peru GDP growth rates (annual, quarterly or monthly) by sector.
</p>
<div/>

In [ ]:
# Call the function to display the window and capture the selected values
selected_spanish, selected_english, sector = show_option_window()

# Display the selected values
print(f"You have selected sector = {sector}, selected_spanish = {selected_spanish}, and selected_english = {selected_english}.")

<div id="select_freq">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">$\bullet$</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Select <code>frequency</code></span></h1>
    </div>

In [ ]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

<div id="counter">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">$\bullet$</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Set counter (dataframe name suffix)</span></h1>
    </div>

In [ ]:
# Call the function to set the counter
if frequency == "monthly":
    counter = 1
elif frequency == "quarterly":
    counter = 2
elif frequency == "annual":
    counter = 2
else:
    counter = None 

print(counter)

<div id="3-1-1">
   <!-- Contenido de la celda de destino -->
</div>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Growth rates datasets concatenation for all frequencies
    </span>
    </h3>

In [ ]:
# Dynamically construct the function name and dictionary name
function_name = f"concatenate_{frequency}_df"
dataframe_dict_name = f"old_dataframes_dict_{counter}"

# Check that both the function and dictionary exist in the global scope
if function_name in globals() and dataframe_dict_name in globals():
    # Call the function using its reference from globals()
    globals()[f"old_{sector}_{frequency}_growth_rates"] = globals()[function_name](
        globals()[dataframe_dict_name], selected_spanish, selected_english
    )
else:
    print(f"Error: {function_name} or {dataframe_dict_name} does not exist in the global scope.")

In [ ]:
#pd.set_option('display.max_rows', None)
globals()[f"old_{sector}_{frequency}_growth_rates"].head(10)

<div id="3-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Uploading data to SQL</span></h3> 

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
Finally, we upload all the datasets generated in this jupyter notebook to the <code>'gdp_revisions_datasets'</code> database of <code>PostgresSQL</code>.
<div/>

In [ ]:
engine = create_sqlalchemy_engine()

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
Loading
<div/>

In [ ]:
globals()[f"old_{sector}_{frequency}_growth_rates"].to_sql(f'old_{sector}_{frequency}_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#select_sector" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#select_sector" style="color: rgb(255, 32, 78); text-decoration: none;">Back to select sectors.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#select_freq" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#select_freq" style="color: rgb(255, 32, 78); text-decoration: none;">Back to select frequency.</a>
</div>

<div id="3-2">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Automatic data creation process in real time: all sectors and frequencies at the same time.
    </span>
    </h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    With this method you can create the dataset for all sectors and all frequencies at the same time. This is more efficient if the goal is to generate all possible combinations of datasets for <code>sector</code> and <code>frequency</code> (without excluding any sector or frequency).
<div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    List of frequencies to be used to create concatenated datasets
    <div/>

In [ ]:
frequencies = [
        "monthly", 
        "quarterly",
        "annual"
    ]

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Function to process growth rates datasets: concatenate and load to SQL
    <div/>

In [ ]:
def process_old_datasets_to_sql(sector, frequency):
    # Initialize counter for loaded DataFrames
    total_loaded = 0

    # Set counter based on frequency
    if frequency == "monthly":
        counter = 1
    elif frequency in ["quarterly", "annual"]:
        counter = 2
    else:
        print(f"Unknown frequency: {frequency}")
        return None

    # Dynamically build function and dictionary names
    function_name = f"concatenate_{frequency}_df"
    dataframe_dict_name = f"old_dataframes_dict_{counter}"

    if function_name in globals() and dataframe_dict_name in globals():
        # Generate the DataFrame
        df_name = f"old_{sector}_{frequency}_growth_rates"
        globals()[df_name] = globals()[function_name](
            globals()[dataframe_dict_name], option_mapping[sector][0], option_mapping[sector][1]
        )

        # Load to SQL
        engine = create_sqlalchemy_engine()
        globals()[df_name].to_sql(df_name, engine, index=False, if_exists='replace')

        return globals()[df_name]
    else:
        print(f"Error: {function_name} or {dataframe_dict_name} does not exist in the global scope.")
        return None

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Run the function to create concatenated datasets for all sectors and frequencies and load to SQL
    <div/>

In [ ]:
# Initialize counter
processed_datasets = 0

# Process all combinations
for sector in option_mapping.keys():
    for frequency in frequencies:
        print(f"Processing {sector} - {frequency}")
        df = process_old_datasets_to_sql(sector, frequency)
        if df is not None:
            display(df.head(10))  # Display the first 10 rows
            processed_datasets += 1  # Increment counter

# Display total number of processed datasets
print(f"Total datasets processed: {processed_datasets}")

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div style="font-size: 16px; background-color: #F5F5F5; padding: 18px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Once you have all the datasets generated by this script (<code>old_gdp_datasets.ipynb</code>) you can concatenate with those generated in the script <code>new_gdp_datasets.ipynb</code>. <b>Section 6</b> of the script <code>aux_files_to_sql.ipynb</code> concatenates both <b>new</b> and <b>old</b> datasets for <b>all sectors</b> and <b>all frequencies</b>.
</div>

---
---
